测试三重过滤分析系统报告

In [1]:
import backtrader as bt
import akshare as ak
import os
import logging
from logging.handlers import TimedRotatingFileHandler
from datetime import datetime
from my_backtrader.tts import TripleScreenTradingSystem

In [2]:
def init_logging():
    """全局日志配置（在策略初始化前调用）"""
    log_format = '%(asctime)s [%(levelname)s] %(message)s'
    log_file = 'logs/futures.log'
    
    # 创建日志目录
    os.makedirs('logs', exist_ok=True)
    
    # 主日志配置
    logging.basicConfig(
        level=logging.INFO,
        format=log_format,
        handlers=[
            TimedRotatingFileHandler(
                log_file, 
                when='D',  # 按天切割
                backupCount=7,
                encoding='utf-8'
            ),
            logging.StreamHandler()
        ]
    )
    
init_logging()

In [3]:
import pandas as pd


def futures_main_weekly_sina(
    symbol: str = "V0",
    start_date: str = "19900101",
    end_date: str = "22220101",
) -> pd.DataFrame:
    """
    新浪财经-期货-主力连续周线数据
    基于日线数据聚合计算，或直接调用周线接口
    """
    # 1. 获取日线数据后聚合为周线（通用方法）
    daily_df = ak.futures_main_sina(symbol=symbol, start_date=start_date, end_date=end_date)
    daily_df["日期"] = pd.to_datetime(daily_df["日期"])
    daily_df.set_index("日期", inplace=True)
    
    # 聚合逻辑：每周一至周五为一周，取首开、最高、最低、尾收、成交量和持仓量总和
    weekly_df = daily_df.resample("W-FRI").agg({
        "开盘价": "first",
        "最高价": "max",
        "最低价": "min",
        "收盘价": "last",
        "成交量": "sum",
        "持仓量": "sum",
        "动态结算价": "last"
    }).dropna()
    
    # 2. 日期筛选
    weekly_df = weekly_df[(weekly_df.index >= pd.to_datetime(start_date)) & 
                         (weekly_df.index <= pd.to_datetime(end_date))]
    weekly_df.reset_index(inplace=True)
    weekly_df.rename(columns={"index": "日期"}, inplace=True)
    return weekly_df

# 示例调用
weekly_data = futures_main_weekly_sina(symbol="V0", start_date="20240101", end_date="20251231")
print(weekly_data)

           日期     开盘价     最高价     最低价     收盘价      成交量      持仓量   动态结算价
0  2024-01-05  5880.0  5953.0  5814.0  5824.0  2755934  3516450  5843.0
1  2024-01-12  5840.0  5867.0  5754.0  5827.0  3376627  4592776  5825.0
2  2024-01-19  5835.0  5904.0  5803.0  5872.0  3078001  4140989  5856.0
3  2024-01-26  5880.0  5970.0  5823.0  5893.0  3442483  3762404  5913.0
4  2024-02-02  5898.0  5954.0  5783.0  5812.0  3252193  3855382  5818.0
..        ...     ...     ...     ...     ...      ...      ...     ...
90 2025-10-03  4894.0  4918.0  4838.0  4839.0  1246327  2147720  4858.0
91 2025-10-10  4825.0  4825.0  4719.0  4735.0  1396656  2361602  4754.0
92 2025-10-17  4730.0  4736.0  4644.0  4688.0  3456442  6243173  4706.0
93 2025-10-24  4696.0  4749.0  4670.0  4708.0  3067461  6033425  4716.0
94 2025-10-31  4708.0  4817.0  4687.0  4701.0  4286043  6040333  4729.0

[95 rows x 8 columns]


In [4]:
class FuturesDataFeed(bt.feeds.PandasData):
    """适配期货数据的Backtrader数据源"""
    params = (
        ('datetime', '日期'),
        ('open', '开盘价'),
        ('high', '最高价'),
        ('low', '最低价'),
        ('close', '收盘价'),
        ('volume', '成交量'),
        ('openinterest', '持仓量'),
    )

In [5]:
symbol_name = "PVC连续"
symbol = 'V0'

daily_df = ak.futures_main_sina(symbol=symbol, start_date='20150101')
weekly_df = futures_main_weekly_sina(symbol=symbol, start_date="20150101")

In [6]:
print(daily_df.head())
print(daily_df.tail())

           日期   开盘价   最高价   最低价   收盘价    成交量    持仓量  动态结算价
0  2015-01-05  5445  5445  5370  5385   6052  35990   5400
1  2015-01-06  5350  5405  5345  5400   5172  35902   5375
2  2015-01-07  5380  5395  5330  5335   7238  36110   5350
3  2015-01-08  5350  5375  5310  5320   5770  37626   5335
4  2015-01-09  5305  5325  5160  5185  23146  43228   5230
              日期   开盘价   最高价   最低价   收盘价      成交量      持仓量  动态结算价
2624  2025-10-27  4708  4757  4694  4746   832377  1211109   4726
2625  2025-10-28  4730  4748  4707  4716   655220  1248205   4727
2626  2025-10-29  4710  4784  4709  4775   840023  1204377   4740
2627  2025-10-30  4775  4817  4754  4766  1045974  1161917   4787
2628  2025-10-31  4762  4768  4687  4701   912449  1214725   4729


In [7]:
print(weekly_df.head())
print(weekly_df.tail())

          日期     开盘价     最高价     最低价     收盘价     成交量     持仓量   动态结算价
0 2015-01-09  5445.0  5445.0  5160.0  5185.0   47378  188856  5230.0
1 2015-01-16  5170.0  5200.0  4915.0  5020.0  100730  249404  4995.0
2 2015-01-23  5065.0  5090.0  4925.0  5010.0   59274  257472  4985.0
3 2015-01-30  4990.0  5000.0  4915.0  4940.0   39318  262878  4955.0
4 2015-02-06  4975.0  5245.0  4970.0  5175.0   65962  206904  5145.0
            日期     开盘价     最高价     最低价     收盘价      成交量      持仓量   动态结算价
549 2025-10-03  4894.0  4918.0  4838.0  4839.0  1246327  2147720  4858.0
550 2025-10-10  4825.0  4825.0  4719.0  4735.0  1396656  2361602  4754.0
551 2025-10-17  4730.0  4736.0  4644.0  4688.0  3456442  6243173  4706.0
552 2025-10-24  4696.0  4749.0  4670.0  4708.0  3067461  6033425  4716.0
553 2025-10-31  4708.0  4817.0  4687.0  4701.0  4286043  6040333  4729.0


In [8]:
daily_df['日期'] = pd.to_datetime(daily_df['日期'])
daily_df = daily_df.sort_values('日期')

weekly_df['日期'] = pd.to_datetime(weekly_df['日期'])
weekly_df = weekly_df.sort_values('日期')

daily_data = FuturesDataFeed(dataname=daily_df)
weekly_data = FuturesDataFeed(dataname=weekly_df)

In [9]:
def run_backtrader_strategy():
    cerebro = bt.Cerebro()

    cerebro.adddata(daily_data, name=symbol_name)
    cerebro.adddata(weekly_data, name=symbol_name)
    cerebro.addstrategy(TripleScreenTradingSystem)
    return cerebro.run()


In [10]:
# 每次调用都会创建新的 cerebro
# results = run_backtrader_strategy()

In [11]:
def run_backtest():
    cerebro = bt.Cerebro()
    
    # 添加策略
    cerebro.addstrategy(TripleScreenTradingSystem)
    
    # 设置初始资金
    cerebro.broker.setcash(100000.0)
    
    # 设置手续费 - 期货手续费较低
    cerebro.broker.setcommission(commission=0.0005)  # 0.05%
    cerebro.adddata(daily_data, name=f"{symbol_name}_DAILY")
    cerebro.adddata(weekly_data, name=f"{symbol_name}_WEEKLY")

    
    print('初始资金: %.2f' % cerebro.broker.getvalue())
    
    # 运行回测
    cerebro.run()
    
    # 绘制图表
    cerebro.plot()
    
    print('最终资金: %.2f' % cerebro.broker.getvalue())

In [12]:
run_backtest()

初始资金: 100000.00
趋势转变! 时间: 2015-08-28, 价格: 5115.00, 类型: 下跌
趋势转变! 时间: 2016-03-18, 价格: 5535.00, 类型: 上涨
趋势转变! 时间: 2017-04-14, 价格: 5665.00, 类型: 下跌
趋势转变! 时间: 2017-08-04, 价格: 7370.00, 类型: 上涨
趋势转变! 时间: 2017-12-22, 价格: 6635.00, 类型: 下跌
趋势转变! 时间: 2018-05-18, 价格: 6730.00, 类型: 上涨
趋势转变! 时间: 2018-11-09, 价格: 6270.00, 类型: 下跌
趋势转变! 时间: 2019-04-26, 价格: 6935.00, 类型: 上涨
趋势转变! 时间: 2019-09-20, 价格: 6500.00, 类型: 下跌
趋势转变! 时间: 2020-07-31, 价格: 6610.00, 类型: 上涨
趋势转变! 时间: 2021-12-31, 价格: 8321.00, 类型: 下跌
趋势转变! 时间: 2023-03-03, 价格: 6435.00, 类型: 上涨
趋势转变! 时间: 2023-05-19, 价格: 5782.00, 类型: 下跌
趋势转变! 时间: 2023-09-15, 价格: 6453.00, 类型: 上涨
趋势转变! 时间: 2023-12-22, 价格: 6094.00, 类型: 下跌
趋势转变! 时间: 2024-05-24, 价格: 6475.00, 类型: 上涨
趋势转变! 时间: 2024-08-23, 价格: 5542.00, 类型: 下跌
趋势转变! 时间: 2025-10-03, 价格: 4839.00, 类型: 上涨
趋势转变! 时间: 2025-10-10, 价格: 4735.00, 类型: 下跌
2015-08-28: 发出做空订单: 价格=5115.00
2015-08-31: 🔍 订单完成检查: size=-1, position=-1
2015-08-31: ✅ 空头开仓完成: 价格=5130.00, entry_bar=163, entry_price=5130.00
2015-08-31: 📋 卖出开空仓完成: 价格=5130.00, 成本=-5130

<IPython.core.display.Javascript object>

最终资金: 93728.06
